# Semantic Memory

In [1]:
# %pip install -U langmem langgraph

In [2]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore

checkpointer = InMemorySaver()
store = InMemoryStore()

In [3]:
agent = create_react_agent("anthropic:claude-3-5-sonnet-latest", tools=[], store=store, checkpointer=checkpointer)

In [4]:
def chat(agent, txt, thread_id):
    result_state = agent.invoke({"messages": [{"role": "user", "content": txt}]}, config={"configurable": {"thread_id": thread_id}})
    return result_state["messages"][-1].content

In [5]:
thread_1 = "thread-1"
chat(agent, "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?", thread_1)

"Here's a 8-week half marathon training plan, assuming you can already run at least 3-4 miles comfortably:\n\nWeek 1-2:\n- Monday: 3 miles easy pace\n- Tuesday: Cross-training/rest\n- Wednesday: 4 miles with intervals (alternate between 1 min fast, 2 min recovery)\n- Thursday: Rest or light cross-training\n- Friday: 3 miles easy pace\n- Saturday: 5-6 miles long run\n- Sunday: Rest or light cross-training\n\nWeek 3-4:\n- Monday: 4 miles easy pace\n- Tuesday: Cross-training/rest\n- Wednesday: 5 miles with tempo runs (middle 2 miles at faster pace)\n- Thursday: Rest or light cross-training\n- Friday: 4 miles easy pace\n- Saturday: 7-8 miles long run\n- Sunday: Rest or light cross-training\n\nWeek 5-6:\n- Monday: 5 miles easy pace\n- Tuesday: Cross-training/rest\n- Wednesday: 6 miles with hills or speed work\n- Thursday: Rest or light cross-training\n- Friday: 4 miles easy pace\n- Saturday: 9-10 miles long run\n- Sunday: Rest or light cross-training\n\nWeek 7-8:\n- Monday: 5 miles easy pac

In [6]:
chat(agent, "Nice! Wish me luck!", thread_1)

"Good luck! You've got this! 💪 Remember to trust your training, pace yourself, and enjoy the journey. Let those endorphins flow and celebrate every milestone along the way. You're going to crush that half marathon! 🏃\u200d♂️🏃\u200d♀️🏅"

In [7]:
thread_2 = "thread-2"
chat(agent, "Nice! Oh thank you! It'll be hard.", thread_2)

'Just trying to help! Let me know if you have any other questions.'

## Adding Memory

We'll use regular tools.

In [8]:
from langmem import create_manage_memory_tool, create_search_memory_tool

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

namespace = ("agent_memories",)
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()
agent = create_react_agent("anthropic:claude-3-5-sonnet-latest", tools=memory_tools, store=store, checkpointer=checkpointer)

/Users/wfh/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langgraph/store/base/embed.py:95: LangChainBetaWarning: The function `init_embeddings` is in beta. It is actively being worked on, so the API may change.
  return init_embeddings(embed)


In [9]:
thread_1 = "thread-1"
chat(agent, "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?", thread_1)

"I've created a comprehensive 2-month training plan for your half marathon preparation. This plan assumes you can already run at least 3 miles comfortably. If you're starting from a lower base, please let me know and we can adjust the plan accordingly.\n\nThe plan includes:\n1. Progressive mileage buildup\n2. Various types of runs (easy, tempo, intervals, long runs)\n3. Rest days and cross-training\n4. Taper period before race day\n5. Important guidelines for pacing and general tips\n\nKey features of this plan:\n- Three running days during the week\n- One long run on weekends\n- Two rest/cross-training days\n- One optional rest/cross-training day\n- Gradual increase in long run distance\n- Proper taper week before the race\n\nWould you like me to:\n1. Adjust the mileage based on your current fitness level?\n2. Provide more specific details about any aspect of the training?\n3. Add specific pace targets based on your current running ability?\n\nAlso, please let me know if you have any 

In [10]:
print(chat(agent, "Nice! Wish me luck! Please note down the detailed memories for me :)", thread_1))

Perfect! I've stored both your detailed training plan and a note about your goal and enthusiasm. Both memories are now saved and can be referenced in future conversations. The training plan includes all the weekly schedules, pacing guidelines, and important tips, while the second memory captures your timeline and positive attitude toward the goal.

Best of luck with your training! Feel free to check back in anytime if you need modifications to the plan or have any questions as you progress through your training journey. Remember to start gradually and listen to your body as you work through the program!


In [11]:
thread_2 = "thread-3"
chat(agent, "Remember what I'm supposed to do for my training this week? It's week 3...", thread_2)

'Yes, I found your training schedule! For Week 3, here\'s what you\'re supposed to do:\n\nMonday: 4-mile easy run\nTuesday: Cross-training or rest\nWednesday: 5-mile run with intervals\nThursday: Rest or light cross-training\nFriday: 4-mile tempo run\nSaturday: 7-mile long run\nSunday: Rest or light walking\n\nRemember:\n- Easy runs should be at a conversational pace\n- Tempo runs should be at a "comfortably hard" pace\n- Long runs should be at an easy, conversational pace\n- Intervals should alternate between hard effort and recovery periods\n- Stay hydrated and listen to your body\n\nWould you like me to remember any specific modifications or notes about how your training is going?'

In [12]:
chat(agent, "That may be tricky. I just sprained my ankle. Could you update my plan to include more cross training? Be sure to update the existing key of our plan", thread_2)

"I've updated your training plan, specifically modifying weeks 3-4 to focus on low-impact activities while your ankle heals. The new plan for this week (Week 3) replaces running with alternative cardio activities:\n\n- Monday: Low-impact cross-training (swimming or cycling)\n- Tuesday: Upper body strength training + Physical therapy exercises\n- Wednesday: Pool running or cycling intervals\n- Thursday: Rest + ankle rehabilitation exercises\n- Friday: Stationary bike or elliptical (30-45 minutes)\n- Saturday: Long session swimming or cycling (45-60 minutes)\n- Sunday: Rest + gentle ankle mobility work\n\nI've also added injury-specific notes to the plan. Remember to:\n1. Consult with a healthcare provider before returning to running\n2. Follow the RICE protocol for your ankle\n3. Only resume running when you're cleared and pain-free\n\nWould you like any clarification about the modified activities or additional adjustments to the plan?"

## Different User Memories

In [13]:
from langmem import create_manage_memory_tool, create_search_memory_tool

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

namespace = ("agent_memories", "{user_id}")
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()

agent = create_react_agent("anthropic:claude-3-5-sonnet-latest", tools=memory_tools, store=store, checkpointer=checkpointer)

In [14]:
def chat(agent, txt, thread_id, user_id):
    result_state = agent.invoke({"messages": [{"role": "user", "content": txt}]}, 
                                config={"configurable": {"thread_id": thread_id, "user_id": user_id}})
    return result_state["messages"][-1].content

In [15]:
thread_1 = "thread-1"
user_id = "User-A"
chat(agent, 
     "Hi I'm Will, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare and help me stay honest??",
     thread_1,
     user_id)

"I've created a progressive 8-week training plan that will help you prepare for your half marathon. To help you stay accountable, you can check in with me regularly, and I'll be able to recall your training plan and progress. I can:\n\n1. Answer questions about specific training days\n2. Help you adjust the plan if needed\n3. Provide tips for specific aspects of training\n4. Track your progress and achievements\n5. Offer motivation and reminders\n\nWould you like me to help you with any specific aspect of the training plan? Also, it would be helpful to know:\n- Your current running experience/fitness level\n- Any previous running injuries or concerns\n- Your target finish time (if you have one)\n\nThis information would help me provide more personalized advice and potentially adjust the training plan to better suit your needs."

In [16]:
thread_1 = "thread-2"
user_id2 = "User-B"
chat(agent, 
     "Hi I'm John, I'm learning chess - could you help me become great??",
     thread_1,
     user_id2)

"I'd be happy to help you on your chess journey! To provide better guidance, it would be helpful to know:\n\n1. What's your current level in chess? (beginner, intermediate, etc.)\n2. Do you already know the basic rules and piece movements?\n3. Are you practicing regularly or just starting out?\n4. Do you have any specific areas of the game you'd like to focus on (openings, tactics, endgame, etc.)?\n\nOnce you share more details about your current chess experience and specific goals, I can provide more targeted advice and help you develop a learning plan. Just let me know, and I'll update my understanding of your chess journey accordingly!"

In [17]:
chat(agent, 
     "Do you remember me liking any sports?",
     thread_1,
     user_id2)

"Based on my memories, I only know that you're interested in chess, which while often considered a mind sport, isn't typically categorized as a traditional sport. I don't have any memories of you mentioning other sports that you like. If you'd like to tell me about any sports you enjoy, I'd be happy to remember that information for future conversations!"

In [18]:
items = store.search(("agent_memories",))
for item in items:
    print(item.namespace, item.value)

('agent_memories', 'User-A') {'content': 'User Will is training for a half marathon in 2 months. Goal event timeline: Approximately May 2024.'}
('agent_memories', 'User-A') {'content': 'Half Marathon Training Plan for Will:\n\nWeek 1-2:\n- Tuesday: 3 miles easy pace\n- Thursday: 3 miles with middle mile at tempo pace\n- Saturday: 4-5 miles long run\n- Other days: Rest or cross-training\n\nWeek 3-4:\n- Tuesday: 4 miles easy pace\n- Thursday: 4 miles with tempo intervals\n- Saturday: 6-7 miles long run\n- Other days: Rest or cross-training\n\nWeek 5-6:\n- Tuesday: 5 miles easy pace\n- Thursday: 5 miles with speed work\n- Saturday: 8-9 miles long run\n- Other days: Rest or cross-training\n\nWeek 7-8:\n- Tuesday: 5-6 miles easy pace\n- Thursday: 5-6 miles with tempo runs\n- Saturday: 10-11 miles long run\n- Other days: Rest or cross-training\n\nFinal 2 weeks:\n- Begin tapering\n- Tuesday: 4 miles easy pace\n- Thursday: 3 miles with light tempo\n- Saturday (week before race): 6-8 miles\n- R

## "Eager" memory retrieval

We can fetch memories before the first LLM call to simplify its response. Otherwise, it has known and unknown unknowns so will almost always try to search for some subclass of questions.

In [19]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langgraph.config import get_store

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

namespace = ("agent_memories",)
memory_tools = [
    create_manage_memory_tool(namespace),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()

def prompt(state):
    # Search over memories based on the messages
    store = get_store()
    items = store.search(namespace, query=state["messages"][-1].content)
    memories = "\n\n".join(str(item) for item in items)
    system_msg = {"role": "system", "content": f"## Memories:\n\n{memories}"}
    return [system_msg] + state["messages"]
    
agent = create_react_agent("anthropic:claude-3-5-sonnet-latest", prompt=prompt, tools=memory_tools, store=store, checkpointer=checkpointer)

In [21]:
thread_1 = "thread-1"
chat(agent, "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?", thread_1, None)

"I've created an 8-week training plan for your half marathon preparation. The plan gradually increases your mileage while incorporating different types of runs and adequate rest periods. Here's a breakdown of the key components:\n\n1. Weekly Structure:\n- 3-4 running days\n- 2 rest/cross-training days\n- 1 long run day (typically Saturday)\n- 1 complete rest or active recovery day (Sunday)\n\n2. Types of Runs:\n- Easy runs: Build base endurance\n- Interval training: Improve speed and running economy\n- Long runs: Build endurance and prepare for race distance\n- Tempo runs: Improve running efficiency\n\n3. Progressive Build-up:\n- Weeks 1-2: Building base mileage\n- Weeks 3-4: Increasing distance\n- Weeks 5-6: Peak training weeks\n- Week 7: Maximum long run\n- Week 8: Taper week for race preparation\n\nThe plan assumes you can already run 3-4 miles comfortably. If this isn't the case, please let me know, and I can adjust the plan accordingly. Also, if you have any specific concerns or c

In [22]:
print(chat(agent, "Nice! Wish me luck! Please note down the detailed memories for me :)", thread_1, None))

I've already stored the detailed training plan in the memories (as shown in the previous interaction), so you'll be able to refer back to it anytime! The memory includes the complete 8-week schedule, including daily workouts and important tips about hydration, rest, and cross-training.

Good luck with your half marathon training! Remember to:
1. Start gradually and follow the progression
2. Listen to your body
3. Stay consistent with your training
4. Keep hydrated and maintain good nutrition
5. Get adequate rest between sessions

Feel free to come back anytime if you need to review the training plan or have questions about specific aspects of your training. The detailed plan will be here in the memories ready for you to access!


In [23]:
thread_2 = "thread-2"
chat(agent, "What I'm supposed to do for my training this week? It's week 3...", thread_2, None)

"Let me check your half marathon training plan. Based on your training schedule for Week 3-4, here's what you should be doing:\n\nWeek 3 Schedule:\n- Monday: 4 mile easy run\n- Tuesday: Cross-training or rest\n- Wednesday: 5 mile run with intervals (2 min fast, 2 min recover)\n- Thursday: Rest or light cross-training\n- Friday: 4 mile easy run\n- Saturday: 7-8 mile long run\n- Sunday: Rest or light walk\n\nRemember:\n- During your intervals on Wednesday, alternate between 2 minutes of faster pace and 2 minutes of recovery pace\n- Your easy runs should be at a comfortable, conversational pace\n- For your weekend long run, you can choose between 7-8 miles depending on how you're feeling\n- Listen to your body and make sure to stay hydrated\n- Cross-training options include swimming, cycling, or strength training\n\nIs there anything specific about any of these workouts you'd like me to explain further?"